In [24]:
# TODO account for main stat of 5-star
# nbd cuz false positives (of good runes) are unlikely:
# they're dragged down by low subs anyways

# TODO multitiered rune sets? Like, instead of certain- and not-, 3 kinds?
# Would need some quick stats (count of my own runes), research, etc
# https://www.youtube.com/watch?v=3D3WOI4-cro
# https://www.youtube.com/watch?v=hshndosfyHo

# TODO remove tricaru team

In [25]:
import numpy as np
import pandas as pd

In [26]:
weights = {'sub_acc': 1, 'sub_res': 1, 'sub_atkp': 1, 'sub_atkf': 0.5, 'sub_defp': 1, 'sub_deff': 0.5, 'sub_hpp': 1, 'sub_hpf': 0.5, 'sub_spd': 2, 'sub_crate': 1, 'sub_cdmg': 1}
weights_off = {'sub_acc': 1, 'sub_res': 0.5, 'sub_atkp': 1, 'sub_atkf': 0.5, 'sub_defp': 0.5, 'sub_deff': 0.25, 'sub_hpp': 0.5, 'sub_hpf': 0.25, 'sub_spd': 2, 'sub_crate': 1, 'sub_cdmg': 1}
weights_def = {'sub_acc': 1, 'sub_res': 1, 'sub_atkp': 1, 'sub_atkf': 0.5, 'sub_defp': 1, 'sub_deff': 0.5, 'sub_hpp': 1, 'sub_hpf': 0.5, 'sub_spd': 2, 'sub_crate': 1, 'sub_cdmg': 1}

In [27]:
df = pd.read_csv('runes-data.csv', sep=';')

In [28]:
df = df.drop(columns=['s1_t',	's1_v',	's1_data',
                      's2_t', 's2_v', 's2_data',
                      's3_t',	's3_v',	's3_data',
                      's4_t',	's4_v',	's4_data',
                      'DT_RowId',	'id',	'unique_id',	'monster',	'originID', 'originName', 'efficiency', 'max_efficiency', 'locked'])
df = df.replace('-', np.nan)


In [29]:
df[df['monster_n']=='Icaru']

,monster_n,ancient,set,slot,grade,level,m_t,m_v,i_t,i_v,sub_res,sub_cdmg,sub_atkf,sub_acc,sub_atkp,sub_defp,sub_deff,sub_hpp,sub_hpf,sub_spd,sub_crate,quality


In [30]:
cols_original = df.columns

In [31]:
# convert inherent stats to eff values, 0-1.
# keep separate because these can't be increased like normal stats

inherent_label_to_sub_label = {'ACC': 'sub_acc', "RES": "sub_res", "ATK%": "sub_atkp", "ATK flat": "sub_atkf",  "DEF%": "sub_defp", "DEF flat": "sub_deff",  "HP%": "sub_hpp", "HP flat": "sub_hpf", "SPD": "sub_spd", "CRate": "sub_crate", "CDmg": "sub_cdmg"}

df['i_t_clean'] = df['i_t'].replace(inherent_label_to_sub_label)

substats_max = {'sub_acc':8, 'sub_res': 8, 'sub_atkp': 8, 'sub_atkf': 20, 'sub_defp': 8, 'sub_deff': 20, 'sub_hpp': 8, 'sub_hpf': 375, 'sub_spd': 6, 'sub_crate': 6, 'sub_cdmg': 7}

df['inh_norm'] = df['i_v'] / (df['i_t_clean'].replace(substats_max)) * (df['i_t_clean'].replace(weights)) 

In [32]:
# convert substats to eff values, 0-5
substats_max = {'sub_acc':8, 'sub_res': 8, 'sub_atkp': 8, 'sub_atkf': 20, 'sub_defp': 8, 'sub_deff': 20, 'sub_hpp': 8, 'sub_hpf': 375, 'sub_spd': 6, 'sub_crate': 6, 'sub_cdmg': 7}

for label in substats_max:
    df[label] = pd.to_numeric(df[label])

for label in substats_max:
    df[label+'_norm'] = df[label]/substats_max[label]*weights[label]


In [33]:
#specify the columns to sum
cols = [str(label+'_norm') for label in substats_max]
cols.append('inh_norm')
#find sum of columns specified 
df['tot_sum_norm'] = df[cols].sum(axis=1)

In [34]:
df['num_powerup_left'] = np.maximum(0, np.subtract(4, np.floor_divide(df['level'], 3)))
df['num_powerup_used'] = np.minimum(4, np.floor_divide(df['level'], 3))


df['num_powerup_incsub'] = df['quality'].map({'Unknown': 0, 'Rare': 2, 'Hero': 3, 'Legend': 4})

df['num_powerup_incsub_left'] = np.maximum(np.subtract(df['num_powerup_incsub'], df['num_powerup_used']) , 0)
df['num_powerup_newsub_left'] = np.subtract(df['num_powerup_left'], df['num_powerup_incsub_left'] )

In [35]:
# From increasing current substats:
# if spd is an increasable substat, then assume all rolls go there
# otherwise assume all rolls go to not-good stats
# TODO this roll could possibly only go to a bad stat (as per norm), not a good one. account for this

df['sub_inc_max_norm'] = (df['num_powerup_incsub_left']*2).where(~df['sub_spd'].isna(), df['num_powerup_incsub_left'])

# assume these are all going to bad stats.
# (even if speed is rollable, probability of going to spd is...low and not worth)
# TODO consider like max vs expected. this straddles the line somewhere
df['sub_new_max_norm'] = df['num_powerup_newsub_left']

In [36]:
# and now sum for the "max" roll eff
cols = ['tot_sum_norm', 'sub_inc_max_norm', 'sub_new_max_norm']

df['tot_max_norm'] = df[cols].sum(axis=1)

In [37]:
# worst cases:
# TODO due to the chance of flat rolls, this is lower.
# also like account for inc rolls can only be present stats; new rolls can only be one of each (not double-up into bad roll)
df['sub_inc_min_norm'] = df['num_powerup_incsub_left'] * 0.5 # (0.5 is worstcase eff from a roll)
df['sub_new_min_norm'] = df['num_powerup_newsub_left'] * 0.5
cols = ['tot_sum_norm', 'sub_inc_min_norm', 'sub_new_min_norm']
df['tot_min_norm'] = df[cols].sum(axis=1)

In [38]:
# for ease in using the outputted sheet
df['is_odd'] = df['slot'].apply(lambda x: (x%2!=0))

certain_set = ['Blade', 'Fatal', 'Rage', 'Swift', 'Violent', 'Will']
df['is_certain_set'] = df['set'].apply(lambda x: x in certain_set)

# To scale to a normalized 0-to-1 scale
# df['tot_max_norm'] = (df['tot_max_norm']/9.round(3)

In [39]:
df['flag'] = False

# Makes sure that runes are in the ratios we want.
# Adjust to get varying quantities of held runes
# Lower factor means holding less runes. But saves mana (since more selective w powerups)
SCALING_FACTOR = 0.9
# try to hit 3000 total ultimately, generally scale as needed
# SCALE SUCH THAT MANA STONES ARE MANAGEABLE    
# GOAL_TOTAL = 3000 # start lower, but steadily raise to 3000.
# TIER_S_RATIO = 0.55
# TIER_A_RATIO = 0.34
# TIER_B_RATIO = 0.11
# IS_ODD_RATIO = 1/3
# CERTAIN_HOLD_FACTOR = 0.9 # all held runes can _potentially_ be better than the n-th best rune.
# what is n divided by the number of total runes of class?
# Note that if scaling_factor=1, then you will hold more runes than the goal_total:
# you'll hold everything already better, and potentially better
# GLOBAL_SUB_CUTOFF = 7 # any non-spd rune, less than this max eff are sold, regardless of the goal totals etc
# To prevent flooding inventory with runes that, even if they're _relatively_ good enough,
# would still be easily, quickly sold by obtaining better runes

# goal: 200
freq = int(200 * SCALING_FACTOR)
mask_subset = (df['is_odd']) & (df['is_certain_set'])
df_subset = df[ mask_subset]
df_subset = df_subset.sort_values(by=['tot_min_norm'], ascending=False)
# if count(df_subset) < goal:
# then enforce eff cutoff instead
# eff_cutoff  = GLOBAL_EFF_CUTOFF
entry = df_subset.iloc[freq-1]
eff_cutoff = entry.tot_sum_norm
print('odd, certain:', eff_cutoff)
df['flag'] = mask_subset & (df['tot_max_norm'] < eff_cutoff) | df['flag']

# goal: 100
freq = int(100 * SCALING_FACTOR)
mask_subset = (df['is_odd']) & (~df['is_certain_set'])
df_subset = df[mask_subset]
df_subset = df_subset.sort_values(by=['tot_min_norm'], ascending=False)
entry = df_subset.iloc[freq-1]
eff_cutoff = entry.tot_sum_norm
print('odd, not certain:', eff_cutoff)
df['flag'] = mask_subset & (df['tot_max_norm'] < eff_cutoff) | df['flag']

# goal: 400
freq = int(400 * SCALING_FACTOR)
freq =320
mask_subset = (~df['is_odd']) & (df['is_certain_set'])
count_spd = sum(mask_subset & (df['m_t'] == 'SPD'))
freq = freq-count_spd
mask_subset = mask_subset & (df['m_t'] != 'SPD')
df_subset = df[mask_subset]
df_subset = df_subset.sort_values(by=['tot_min_norm'], ascending=False)
entry = df_subset.iloc[freq-1]
eff_cutoff = entry.tot_sum_norm
print('even, certain:', eff_cutoff)
df['flag'] = mask_subset & (df['tot_max_norm'] < eff_cutoff) | df['flag']

# goal: 200
freq = int(200 * SCALING_FACTOR)
mask_subset = (~df['is_odd']) & (~df['is_certain_set'])
count_spd = sum(mask_subset & (df['m_t'] == 'SPD'))
freq = freq-count_spd
mask_subset = mask_subset & (df['m_t'] != 'SPD')
df_subset = df[mask_subset]
df_subset = df_subset.sort_values(by=['tot_min_norm'], ascending=False)
entry = df_subset.iloc[freq-1]
eff_cutoff = entry.tot_sum_norm
print('even, not certain:', eff_cutoff)
df['flag'] = mask_subset & (df['tot_max_norm'] < eff_cutoff) | df['flag']

odd, certain: 5.958333333333333
odd, not certain: 7.430666666666666
even, certain: 4.125
even, not certain: 7.0


In [40]:
pd.options.display.max_columns = None
df.head(20)
# df[df['monster_n']=='Okeanos']

,monster_n,ancient,set,slot,grade,level,m_t,m_v,i_t,i_v,sub_res,sub_cdmg,sub_atkf,sub_acc,sub_atkp,sub_defp,sub_deff,sub_hpp,sub_hpf,sub_spd,sub_crate,quality,i_t_clean,inh_norm,sub_acc_norm,sub_res_norm,sub_atkp_norm,sub_atkf_norm,sub_defp_norm,sub_deff_norm,sub_hpp_norm,sub_hpf_norm,sub_spd_norm,sub_crate_norm,sub_cdmg_norm,tot_sum_norm,num_powerup_left,num_powerup_used,num_powerup_incsub,num_powerup_incsub_left,num_powerup_newsub_left,sub_inc_max_norm,sub_new_max_norm,tot_max_norm,sub_inc_min_norm,sub_new_min_norm,tot_min_norm,is_odd,is_certain_set,flag
0,Inventory,False,Energy,1,6,6,ATK flat,70,RES,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,5.0,Rare,sub_res,0.750000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.333333,0.833333,NaN,6.916667,2,2,2.0,0.0,2.0,0.0,2.0,8.916667,0.0,1.0,7.916667,True,False,False
1,Inventory,False,Energy,1,6,6,ATK flat,70,CRate,6,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0,NaN,Rare,sub_crate,1.000000,NaN,0.625,NaN,NaN,NaN,NaN,NaN,NaN,5.000000,NaN,NaN,6.625000,2,2,2.0,0.0,2.0,0.0,2.0,8.625000,0.0,1.0,7.625000,True,False,False
2,Homunculus - Hypernox,False,Energy,1,6,12,ATK flat,118,CRate,5,NaN,NaN,NaN,11.0,9.0,NaN,NaN,17.0,NaN,22.0,NaN,Legend,sub_crate,0.833333,1.375,NaN,1.125,NaN,NaN,NaN,2.125,NaN,7.333333,NaN,NaN,12.791667,0,4,4.0,0.0,0.0,0.0,0.0,12.791667,0.0,0.0,12.791667,True,False,False
3,Diana,False,Energy,1,6,9,ATK flat,94,NaN,0,NaN,NaN,NaN,11.0,13.0,NaN,NaN,15.0,NaN,5.0,NaN,Legend,NaN,NaN,1.375,NaN,1.625,NaN,NaN,NaN,1.875,NaN,1.666667,NaN,NaN,6.541667,1,3,4.0,1.0,0.0,2.0,0.0,8.541667,0.5,0.0,7.041667,True,False,False
4,Inventory,True,Energy,2,5,12,SPD,29,NaN,0,NaN,NaN,23.0,NaN,NaN,16.0,NaN,9.0,547.0,NaN,NaN,Legend,NaN,NaN,NaN,NaN,NaN,0.575,2.000,NaN,1.125,0.729333,NaN,NaN,NaN,4.429333,0,4,4.0,0.0,0.0,0.0,0.0,4.429333,0.0,0.0,4.429333,False,False,False
5,Inventory,True,Energy,2,5,6,SPD,17,NaN,0,NaN,NaN,NaN,NaN,NaN,16.0,NaN,NaN,NaN,NaN,7.0,Rare,NaN,NaN,NaN,NaN,NaN,NaN,2.000,NaN,NaN,NaN,NaN,1.166667,NaN,3.166667,2,2,2.0,0.0,2.0,0.0,2.0,5.166667,0.0,1.0,4.166667,False,False,False
6,Inventory,False,Energy,2,6,6,SPD,19,ACC,8,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,395.0,NaN,NaN,Rare,sub_acc,1.000000,NaN,1.250,NaN,NaN,NaN,NaN,NaN,0.526667,NaN,NaN,NaN,2.776667,2,2,2.0,0.0,2.0,0.0,2.0,4.776667,0.0,1.0,3.776667,False,False,False
7,Inventory,True,Energy,2,5,9,SPD,23,DEF flat,14,NaN,11.0,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,Hero,sub_deff,0.350000,NaN,NaN,NaN,0.500,NaN,NaN,NaN,NaN,NaN,1.500000,1.571429,3.921429,1,3,3.0,0.0,1.0,0.0,1.0,4.921429,0.0,0.5,4.421429,False,False,False
8,Inventory,False,Energy,2,6,9,SPD,25,ATK%,5,NaN,NaN,NaN,NaN,NaN,21.0,26.0,5.0,NaN,NaN,NaN,Hero,sub_atkp,0.625000,NaN,NaN,NaN,NaN,2.625,0.65,0.625,NaN,NaN,NaN,NaN,4.525000,1,3,3.0,0.0,1.0,0.0,1.0,5.525000,0.0,0.5,5.025000,False,False,False
9,Anavel,False,Energy,2,5,15,SPD,39,HP%,4,13.0,5.0,NaN,17.0,11.0,NaN,NaN,NaN,NaN,NaN,NaN,Legend,sub_hpp,0.500000,2.125,1.625,1.375,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.714286,6.339286,0,4,4.0,0.0,0.0,0.0,0.0,6.339286,0.0,0.0,6.339286,False,False,False


In [41]:
cols_export = cols_original
cols_export = cols_export.append(pd.Index(['tot_sum_norm', 'tot_min_norm', 'tot_max_norm', 'is_odd', 'is_certain_set', 'num_powerup_incsub_left', 'flag']))
df_export = df[cols_export]

In [42]:
df_export = df_export.sort_values(['set', 'grade', 'slot'], ascending=[True, False, True])
# df_export = df_export.sort_values('grade', ascending=False)
# df_export = df_export.sort_values('set')

In [43]:
# using datetime module
import datetime
  
# ct stores current time
ct = datetime.datetime.now()
ct = int(ct.timestamp())
df_export.to_csv(f'runes-data-out-{str(ct)}.csv', sep='\t')

In [44]:
df_export[df_export['flag']]

,monster_n,ancient,set,slot,grade,level,m_t,m_v,i_t,i_v,sub_res,sub_cdmg,sub_atkf,sub_acc,sub_atkp,sub_defp,sub_deff,sub_hpp,sub_hpf,sub_spd,sub_crate,quality,tot_sum_norm,tot_min_norm,tot_max_norm,is_odd,is_certain_set,num_powerup_incsub_left,flag
953,Inventory,False,Blade,5,6,0,HP flat,360,NaN,0,NaN,NaN,NaN,NaN,7.0,NaN,13.0,NaN,NaN,NaN,4.0,Hero,1.866667,3.866667,5.866667,True,True,3.0,True
1024,Inventory,False,Despair,5,6,0,HP flat,360,NaN,0,NaN,7.0,NaN,NaN,NaN,NaN,19.0,NaN,NaN,NaN,NaN,Rare,1.475000,3.475000,5.475000,True,False,2.0,True
781,Inventory,False,Destroy,2,6,9,DEF%,38,SPD,6,NaN,11.0,NaN,NaN,NaN,NaN,40.0,6.0,NaN,NaN,NaN,Hero,5.321429,5.821429,6.321429,False,False,0.0,True
785,Chiwu,True,Destroy,4,5,9,HP%,29,NaN,0,NaN,NaN,26.0,5.0,NaN,NaN,NaN,NaN,NaN,13.0,NaN,Hero,5.608333,6.108333,6.608333,False,False,0.0,True
825,Inventory,False,Determination,4,6,15,DEF%,63,CRate,6,8.0,11.0,55.0,NaN,NaN,NaN,NaN,NaN,NaN,6.0,NaN,Hero,6.946429,6.946429,6.946429,False,False,0.0,True
832,Inventory,False,Determination,5,6,12,HP flat,1800,NaN,0,7.0,NaN,NaN,NaN,NaN,12.0,79.0,NaN,NaN,NaN,5.0,Hero,5.183333,5.183333,5.183333,True,False,0.0,True
827,Inventory,False,Determination,6,6,15,DEF%,63,NaN,0,NaN,NaN,17.0,15.0,NaN,NaN,NaN,NaN,229.0,12.0,NaN,Rare,6.605333,6.605333,6.605333,False,False,0.0,True
1007,Inventory,False,Endure,5,6,1,HP flat,480,SPD,4,NaN,NaN,NaN,NaN,NaN,NaN,13.0,NaN,NaN,NaN,6.0,Rare,2.658333,4.658333,6.658333,True,False,2.0,True
913,Inventory,False,Energy,3,6,0,DEF flat,22,CDmg,6,8.0,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,Rare,2.732143,4.732143,6.732143,True,False,2.0,True
23,Inventory,False,Energy,4,6,9,CDmg,47,HP flat,235,NaN,NaN,NaN,6.0,17.0,NaN,NaN,NaN,NaN,8.0,NaN,Hero,5.855000,6.355000,6.855000,False,False,0.0,True


In [45]:
# TODO custom sort keys??
# TODO col of "reapp friendly"